# Create Figures for Birnstiel et al. 2015
This script loads the data and (re-)creates the figures for the paper

> T. Birnstiel, S. M. Andrews, P. Pinilla, and M. Kama  
> *Dust Evolution Can Produce Scattered Light Gaps in Protoplanetary Disks*  
> Astrophysical Journal Letters, 2015

# Imports

In [ ]:
import glob, os, aux_functions, fig1, h5py

import numpy             as np
import matplotlib        as mpl
import matplotlib.pyplot as plt

from matplotlib        import ticker, gridspec
from matplotlib.colors import LogNorm
from aux_functions     import AU, Grav, k_b, mu, m_p
    
import distribution_reconstruction

%matplotlib

colors   = ['#de2d26','#756bb1']
contcols = ['#e6550d','#08519c','#31a354']

# Load Input Data

In [ ]:
with h5py.File('data.hdf5') as fid:
    sim_file  = fid['sim_file'][()]  # Name of the simulation file
    radmc_dir = fid['radmc_dir'][()] # Name of the RADMC-3D setup
    time      = fid['time'][()]      # Time of the snapshot                 [s]
    sig_da    = fid['sig_da'][()]    # Dust surface densities \Sigma_d(r,a) [g cm^-2]
    sig_g     = fid['sig_g'][()]     # Gas  surface densities \Sigma_g(r)   [g cm^-2]
    M_star    = fid['M_star'][()]    # Mass of central star                 [g]
    v_gas     = fid['v_gas'][()]     # Gas radial velocities                [cm s^-1]
    v_dust    = fid['v_dust'][()]    # Dust radial velocities v_d(r,a)      [cm s^-1]
    rho_s     = fid['rho_s'][()]     # Material density of the grains       [g cm^-3]
    r         = fid['r'][()]         # Radial grid                          [cm]
    a         = fid['a'][()]         # Grain size grid                      [cm]
    T         = fid['T'][()]         # Temperature profile in simulation    [K]
    v_f       = fid['v_f'][()]       # Fragmentation threshold velocity     [cm s^-1]
    alpha     = fid['alpha'][()]     # Turbulence parameter \alpha(r)       [-]
    gamma     = fid['gamma'][()]     # Radial pressure exponent dlnP/dlnr   [-]
    lambdas   = fid['lambdas'][()]   # Wavlengths of the images/profiles    [cm]
    a_0       = fid['a_0'][()]       # Initial maximum particle size        [cm]
#
# calculate derived quantities
#
sig_d  = sig_da.sum(0)               # Total dust surface density    [g cm^-2]
eps    = sig_d/sig_g                 # Dust-to-gas mass ratio        [-]
om     = np.sqrt(Grav*M_star/r**3)   # Keplerian frequency           [s^-1]
v_k    = om*r                        # Keplerian Velocity            [cm s^-1]
cs     = np.sqrt(k_b*T/mu/m_p)       # Isothermal sound speed        [cm s^-1]

# Figure 1
Figure 1 is independent of the others, we will just run the script to create the figure in the background.

We will set our plotting preferences afterwards to avoid `fig1.py` overwriting them.

In [ ]:
fig1.main();
aux_functions.better_plots(sans=False,fs=16,lw=1.5)

# Figure 2
### Get Opacities
Get opacities from RADMC-3D run. Assuming that the alphabetically sorting corresponds to wavelengths sorting.

In [ ]:
LAM  = None
CABS = []
CSCA = []
for _f in  glob.glob(radmc_dir+os.sep+'dustkap*.inp'):
    with open(_f) as fid:
        line    = fid.readline().strip()
        while line[0]=='#': line=fid.readline().strip()
        iformat = int(np.loadtxt([line],dtype='int',delimiter=' '))
        nf      = np.fromfile(fid,dtype='int',count=1,sep=' ')[0]
        _data   = np.loadtxt(fid)
        _lam    = _data[:,0]
        CABS   +=[_data[:,1]]
        CSCA   +=[_data[:,2]]
        
        if LAM is None:
            LAM = _lam*1e-4
        else:
            if any(LAM!=_lam*1e-4):
                print(_f+': wavelengths do not match!')
#
# fill inn the large grain opacities as zeros (they were omitted because of low density)
#
for i in range(len(a)-len(CABS)):
    CABS += [np.zeros(len(LAM))]
    CSCA += [np.zeros(len(LAM))]
CABS = np.array(CABS)
CSCA = np.array(CSCA)
#
# interpolate at the desired wavelengths
#
kappa_abs = np.zeros([len(a),len(lambdas)])
kappa_sca = np.zeros([len(a),len(lambdas)])
for il in range(len(lambdas)):
    for ia in range(len(a)):
        kappa_abs[ia,il] = np.interp(lambdas[il],LAM,CABS[ia,:])
        kappa_sca[ia,il] = np.interp(lambdas[il],LAM,CSCA[ia,:])

### Call the routine to reconstruct the size-distribution

In [ ]:
sig_dr,a_max,r_f,sig_1,sig_2,sig_3 = distribution_reconstruction.reconstruct_size_distribution(\
    r,a,time,sig_g,\
    sig_d,alpha,rho_s,T,M_star,v_f,a_0=a_0,fix_pd=2.5)
print('r_f = {} AU'.format(r_f/AU))

### Plotting

In [ ]:
vmin,vmax=-9,2
#
# define the figure layout
#
f=plt.figure(figsize=(5.5,6.6))

gs = gridspec.GridSpec(2,2,width_ratios=[10,0.6])
gs.update(wspace=0.0,hspace=0.075,top=0.98,bottom=0.09,right=0.85)
ax1=plt.subplot(gs[0,0])
ax2=plt.subplot(gs[1,0])
#
# factor for converting to dust density distribution
#
gsf = 2*(a[1]/a[0]-1)/(a[1]/a[0]+1)
#
# first plot - simulated distribution
#
c   = ax1.contourf(r/AU,np.log10(a),np.maximum(np.minimum(vmax,sig_da/gsf),vmin),10.**np.arange(vmin,vmax+1),cmap='Greys',norm=LogNorm())
cax = plt.subplot(gs[0,1:])
cb  = plt.colorbar(c,cax=cax)
cb.set_label('$a\cdot\Sigma_\mathrm{d}(r,a)$ [g cm$^{-2}$]')
#
# first panel - overlay reconstruction
#
for i in range(3):
    #
    # mask the regions
    #
    if i == 0:
        mask = (sig_1<sig_2) | (sig_1<=sig_3) | (sig_1<10.**vmin)
    if i == 1:
        mask = (sig_2<sig_1) | (sig_2<=sig_3) | (sig_2<10.**vmin)
    if i == 2:
        mask = (sig_3<sig_1) | (sig_3<=sig_2) | (sig_3<10.**vmin)
    sig  = np.ma.masked_array(sig_dr, mask)
    #
    # plot the mesh
    #
    c=ax1.contour(r/AU,np.log10(a),sig/gsf,vmin=10.**vmin,vmax=10.**vmax,lw=1.5,colors=contcols[i],alpha=1.0,norm=LogNorm())
#
# second panel: optical depth comparison
#
x_text  = [1.1,0.8]
ha_text = ['left','right']
va_text = ['bottom','top']
for il in range(len(lambdas)):
    tau_sim = np.sum((kappa_abs[:,il][:,np.newaxis]+kappa_sca[:,il][:,np.newaxis])*sig_da,0)
    tau_rec = np.sum((kappa_abs[:,il][:,np.newaxis]+kappa_sca[:,il][:,np.newaxis])*sig_dr,0)

    ax2.semilogx(r/AU,np.log10(tau_sim),'-', c=colors[il])
    ax2.semilogx(r/AU,np.log10(tau_rec),'--',c=colors[il])
    
    if lambdas[il]<0.1:
        txt='$\lambda = $ {:0.4g} $\mu$m'.format(lambdas[il]*1e4)
    else:
        txt='$\lambda = $ {:0.4g} mm'.format(lambdas[il]*10)
        
    ax2.text(x_text[il]*r_f/AU,np.log10(tau_sim[abs(r-r_f).argmin()]),txt,ha=ha_text[il],va=va_text[il],color=colors[il],fontsize='small')
        
ax2.set_ylabel(u'log optical depth');
ax2.set_ylim(-2,4);
#
# add fake legend
#
ax2.plot([0],[0],'k-', label='simulation')
ax2.plot([0],[0],'k--',label='reconstructed $\,$')
ax2.legend(fontsize='small').get_frame().set_alpha(0);
#
# formatting
#
for ax in [ax1]:
    ax.loglog(r/AU,np.log10(a_max),c='w',ls='--',lw=2)
    ax.set_yscale('linear')
    ax.set_ylim(np.log10(a[[0,-1]]))
    ax.set_ylabel('log grain size [cm]')

for ax in [ax1,ax2]:
    ax.plot(r_f/AU*np.ones(2),ax.get_ylim(),c='k',ls=':',lw=2)
    ax.set_xlim(0.3,750)
    ax.set_xscale('log')
    ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%g'))
    
ax1.set_xticklabels('')

ax1.text(0.04, 0.91,'a)',transform=ax1.transAxes)
ax2.text(0.04, 0.91,'b)',transform=ax2.transAxes)
    
ax2.set_xlabel('radius [AU]')

f.savefig('fig2.pdf',facecolor='None')

# Figure 3

Set image parameters

In [ ]:
img_name    = 'paperimage'
#
# other parameters
#
rmax_AU     = 200.              # plotting/image range in AU
rmax_AU_z   = 200.              # zoomed in plotting range in AU, no zoom if equal to rmax_AU
npix        = 2048              # nr of pixels on each side of the image
AU_tickpos  = np.array([0.2,0.5,0.75,1.0])*10.**round(np.log10(rmax_AU))
dpc_img     = 54.               # assumed distance of image
rms         = 5e-5              # assumed noise level [Jy/arcsec]
dynrng      = 3                 # how many orders of magnitude above noise
rmax_as     = rmax_AU/dpc_img   # plotting range in arcsec
rmin_as     = 0.01              # minimum radius for plotting and scaling in arsec
images_done = False

Calculate images (takes a while) -- will only be done once, or when `images_done=False` is reset.

In [ ]:
imgs = []
if images_done:
    print('Already calculated the images before, skipping it.')
    
for lam in lambdas:
    for suffix,rmx in zip(['','_z'],[rmax_AU,rmax_AU_z]):
        img = img_name+'_{:0.4g}micron'.format(lam*1e4)+suffix
        if suffix=='_z' and rmax_AU==rmax_AU_z: continue
        if not images_done:
            aux_functions.makeimage(lam,dirname=radmc_dir,\
                incl=10.0,PA=0.0,npix=npix,sizeau=2*rmx,\
                img_name=img,dpc=dpc_img,secondorder='');
        imgs +=[radmc_dir+os.sep+img+'.fits']
images_done = True

Read in the image data.

In [ ]:
image_data = []
for i in imgs:
    image_data += [aux_functions.get_image_data(i)]

### Plotting

In [ ]:
#
# set up figure and axes
#
plt.figure(figsize=(5.5,6.6))

gs  = mpl.gridspec.GridSpec(2,2,height_ratios=[0.8,1])

ax1 = plt.subplot(gs[0,0])
ax2 = plt.subplot(gs[0,1])
ax3 = plt.subplot(gs[1,:])

ax1.set_aspect(1)
ax2.set_aspect(1)

gs.update(wspace=0.0,hspace=0.2,top=0.95,bottom=0.1,left=0.2,right=0.95)

plt.setp(ax2.get_yticklabels(), visible=False)
#
# Images
#
for i,ax in enumerate([ax1,ax2]):
    x,y,Z,img_lam,h = image_data[i]
    X,Y    = np.meshgrid(x,y)
    X_AU   = X*dpc_img
    Y_AU   = Y*dpc_img
    R_AS   = np.sqrt(X**2+Y**2)
    
    if i == 0:
        scale = (X**2+Y**2)
        txt   = r'$\lambda = {:0.4g}$ $\mu$m'.format(img_lam*1e4)
        cm    = 'Reds_r'
    else:
        scale  = 1
        txt   = r'$\lambda = {:0.4g}$ mm'.format(img_lam*10)
        cm    = 'Purples_r'
    
    
    vmin,vmax = 7e-4,9e-2
    
    mappable=ax.imshow(scale*Z,vmin=vmin,vmax=vmax,\
                norm=mpl.colors.LogNorm(),\
                extent=[X_AU.min(),X_AU.max(),Y_AU.min(),Y_AU.max()],\
                cmap=cm,interpolation='nearest',origin='lower')

    ax.set_xlim(-rmax_AU,rmax_AU);
    ax.set_ylim(-rmax_AU,rmax_AU);
    
    ax.set_xlabel('$x$ [AU]',fontsize='small');
    ax.set_ylabel('$y$ [AU]',fontsize='small');
    
    ax.text(0.1,0.85,txt,transform=ax.transAxes,color='w',fontsize='small')
        
    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=4,prune='both'))
    ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=4))
    ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator())


    plt.setp(ax.get_yticklabels()[0],visible=False)
    plt.setp(ax.get_yticklabels()[-1],visible=False)

ax2.yaxis.get_label().set_visible(False)
#
# ------------
# profile plot
# ------------
#
for i,img_arr in enumerate(image_data):
    x,y,img,img_lam,h = img_arr
    x0      = h['CRPIX1']
    y0      = h['CRPIX2']
    profile = img[x0:,y0]
    r_as    = x[x0:]
    r_au    = r_as*dpc_img
    
    if i == 0:
        txt   = r'$\lambda = {:0.4g}$ $\mu$m'.format(img_lam*1e4)
        yfct  = 0.17
        #
        # find minimum between 1 and 100
        #
        mask = (r_au>1) & (r_au<100)
        rmin = r_au[mask][profile[mask].argmin()]
        imin = abs(r_au-rmin).argmin()
        ax3.annotate(xy=(rmin,np.log10(profile[imin])),xytext=(rmin,np.log10(5*profile[imin])),color='k',s='dip',ha='center',arrowprops=dict(lw=0.5,fc='k',arrowstyle = 'simple,head_width=.5,head_length=.25',connectionstyle = 'arc3,rad=0'))
    else:
        txt   = r'$\lambda = {:0.4g}$ mm'.format(img_lam*10)
        yfct  = 1.5
    
    l,=ax3.semilogx(r_au,np.log10(profile),label=r'$\lambda = {:0.4g}$ micron'.format(img_lam*1e4),c=colors[i])
    
    labelpos=40
    ax3.text(labelpos,np.log10(yfct*profile[abs(r_au-labelpos).argmin()]),txt,color=l.get_color(),fontsize='small')   
#
# formatting the profile plot
#
ax3.set_xlim(1,r_au[-1])
ax3.set_ylim(-4.5,1)
ax3.add_artist(mpl.lines.Line2D(r_f/AU*np.ones(2),ax3.get_ylim(),color='k',ls=':'))
ax3.set_xlabel('radius [AU]')
ax3.set_ylabel('log Intensity [Jy/arcsec$^2$]')
ax3.xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%g'))
ax3.tick_params(axis='y',which='minor',left='off',right='off')
plt.savefig('fig3.pdf',facecolor='None')

# Figure 4

In [ ]:
alpha_mix   = 1./(om*time) # lower limit
alpha_sweep = 1./(om*time)**2/eps*(cs/om/r)**-2*0.55/abs(gamma) # lower
alpha_frag  = abs(gamma)/(3*eps)*(v_f/v_k)**2# upper limit

f  = plt.figure(figsize=(5,4))
ax = plt.gca()
ax.loglog(r/AU,alpha_mix,  label='lower limit: mixing')
ax.loglog(r/AU,alpha_sweep,label='lower limit: sweep-up')
ax.loglog(r/AU,alpha_frag, label='upper limit: fragmentation')
ax.fill_between(r/AU,alpha_frag,np.maximum(alpha_sweep,alpha_mix),alpha=0.5,color=plt.rcParams['axes.color_cycle'][4])
ax.loglog(r/AU,alpha,'k--')

ax.set_ylim(1e-6,1e-1)
ax.set_xlim(0.3,750)

ax.set_xlabel('radius [AU]')
ax.set_ylabel(r'$\alpha_\mathrm{t}$')
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%g'))
ax.legend(loc='best',fontsize='x-small')
plt.tight_layout()
f.savefig('fig4.pdf',facecolor='None')